<a href="https://colab.research.google.com/github/arberas96/Code/blob/master/Final_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install nx_altair #COPIA
!pip install gender_guesser
!pip install python-Levenshtein

In [0]:
import pandas as pd
import numpy as np
import scipy as spy
from collections import Counter
from Levenshtein import distance as lev_distance
from Levenshtein import ratio
import gender_guesser.detector as gender
import nltk
import math
import networkx as nx
import nx_altair as nxa
import altair as alt
from networkx.drawing.nx_pydot import write_dot
import random
import geopy.distance
from datetime import datetime
from datetime import date
from nx_altair.core import to_pandas_nodes, to_pandas_edges_arrows, to_pandas_edges
from nx_altair.draw_altair import draw_networkx_nodes, draw_networkx_labels, draw_networkx_arrows
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')

In [0]:
from google.colab import files
files.upload()

In [0]:
depositions_df=pd.read_json('all_depositions_1.json')

In [0]:
depositions_df

In [0]:
#Añadimos un asterisco, *, en caso de no haber 'forename' o 'surname'
#for i in range(len(depositions_df)):
#  for persona in depositions_df.iloc[i]['people_list']:
#    if 'forename' not in persona.keys():
#      persona['forename']='None'
#    if 'surname' not in persona.keys():
#      persona['surname']='None'

In [0]:
#for i in range(len(depositions_df)):
#  for persona in depositions_df.iloc[i]['people_list']:
#    nombre_sep=persona['forename'].split()
#    nombre=nombre_sep[0]
#    if len(nombre_sep)>1:
#      for palabra in range(1,len(nombre_sep)):
#        nombre=nombre+' '+nombre_sep[palabra]
#    persona['forename']=nombre



In [0]:
#for i in range(4880,len(depositions_df)):
#  for persona in depositions_df.iloc[i]['people_list']:
#    if 'sex' not in persona.keys() or int(persona['sex'])==3:
#      sexo_estim=gender.Detector().get_gender(persona['forename'])
#      if sexo_estim=='male' or sexo_estim=='mostly_male':
#        sexo_estim=1
#      elif sexo_estim=='female' or sexo_estim=='mostly_female':
#        sexo_estim=2
#      else:
#        sexo_estim=3
#      persona['sex']=sexo_estim

Funciones auxiliares:

In [0]:
#Conseguimos indx2person, person2indx, indx2role:

name_counter = Counter()
people_list = depositions_df['people_list'].tolist()
names_list = []
person2indx = {}
#All participants have only one role (checked)
indx2role = {}
for i, dep_part in enumerate(people_list):
    names = []
    for person in dep_part:
        person_str = ''
        if 'forename' in person:
            person_str += person['forename']
        else:
            person_str += 'None'
        person_str += ' '
        
        if 'surname' in person:
            person_str += person['surname']
        else:
            person_str += 'None'
    
        if person_str not in person2indx:
            idx = len(person2indx)
            person2indx[person_str] = idx
            indx2role[idx] = {i : person['role']}
        else:
            idx = person2indx[person_str]
            indx2role[idx][i] = person['role']
        name_counter[person_str] += 1
        names.append(person_str)
    names_list.append(names)

indx2person = {v:k for k,v in person2indx.items()}

In [0]:
#Similitud de Jaccard:

def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    if len(s1.union(s2))!=0:
      return len(s1.intersection(s2)) / len(s1.union(s2))

In [0]:
#Union de listas sin repetición:

def union(lista_1,lista_2):
  in_first = set(lista_1)
  in_second = set(lista_2)

  in_second_but_not_in_first = in_second - in_first
  return lista_1 + list(in_second_but_not_in_first)
 

In [0]:
contador=0
for i in range(len(indx2role)):
  for j in range(len(indx2role[i])):
    contador+=1


Primera variable: $G_{i,j}$

In [0]:
#Vamos a definir una función que saque todos los participantes que coinciden con cierta persona, dada por su index.
#Entrada: indx de la persona
#Salida: lista de todas las personas con las que coincide en los distintos testimonios

#Aquí estamos sacando als personas que coinciden en todas las deposition para cada índice. Sin embargo, lo que nos interesa es saber para cada índice el número de personas.
#Lo único que necesitamos, por lo tanto, es una función que nos devuelva la lista de participantes de una deposition determinada. (Para después trabajar con ello y sacar el índice de Jaccard.)

#Tenemos dos funciones: participantes_no_confirm y participantes_confirm:

# -participantes_no_confirm: Si tenemos una persona que no sabemos si es la misma que otra, quiere decir que esa persona en principio se supone que solo aparece en una deposition. Por lo tanto,
# las personas con las que va a aparecer son las mismas que con las que aparece en dicha deposition. Por lo que, únicamente nos hace falta saber la deposition de la persona que estamos analizando.

# -participantes_confirm: Esta nos da todas las personas en las que aparece la persona que estamos analizando. Cuando creemos nuestros grupos de personas confirmadas tendrán que ser un diccionario similar
# al de indx2role, pero cada persona tendrá asociadas las deposition en las que se repite su aparición. (Es decir, cuando tengas esto creado, en caso de ir encontrando gente que se parece, vas a tener 
# que cambiar en esta función donde aparece indx2role por el nombre del conjunto donde aparezcan todos esos grupos de gente confirmada.)

def participantes_no_confirm(deposition):
  participantes=list()
  for person in depositions_df.iloc[deposition]['people_list']:
    if 'forename' in person.keys() and 'surname' in person.keys(): #No se si es necesario, ya que creo que en caso de no tener nombre o apellido, se representaría con un asterisco.
      persona=person['forename']+' '+person['surname']
    elif 'forename' in person.keys() and 'surename' not in person.keys():
      persona=person['forename']
    elif 'forename' not in person.keys() and 'surename' in person.keys():
      persona=person['surename']
    if persona not in participantes:
      participantes.append(persona)
  return participantes


#Cuando tengamos una persona "confirmada", entonces lo que tendremos será una persona que aparece en más de una deposition. En ese momento se tendrá que usar esta función.

def participantes_confirm(persona):
  if isinstance(persona,str):
    persona=person2indx[persona]
  participantes=list()
  for deposition in indx2role[persona].keys(): #En el momento en el que tengamos un grupo confirmado, vamos a poder generar un diccionario de diccionarios, similar a indx2role, pero con las deposition en las que aparece ciero índice.
    for person in depositions_df.iloc[deposition]['people_list']:
      if 'forename' in person.keys() and 'surname' in person.keys(): #No se si es necesario, ya que creo que en caso de no tener nombre o apellido, se representaría con un asterisco.
        persona=person['forename']+' '+person['surname']
      elif 'forename' in person.keys() and 'surename' not in person.keys():
        persona=person['forename']
      elif 'forename' not in person.keys() and 'surename' in person.keys():
        persona=person['surename']
      if persona not in participantes:
        participantes.append(persona)
  return participantes





In [0]:
### Aquí se calcula el índice de Jaccard de los participantes coincidentes de ambas personas que estamos analizando.
### Importante: Acuerdate de que cuando crees los grupos confirmados, tienes que cambiar la función de participantes_confirm.

## Nota: esta función se puede hacer variable en cuanto a los argumentos. Una idea sería poner todos como None, o un valor neutral, para que el usuario o quien vaya a usar dicha función solo meta lo que necesita.
## Es decir, que se introduzca la deposition de una persona en caso de no estar confirmada o el nombre de una persona en caso de ser confirmada.

#def G(persona_1=None, deposition_1=None, confirm_1=False, persona_2=None, deposition_2=None, confirm_2=False):
def GG(deposition_1,deposition_2):
  participantes_1=participantes_no_confirm(deposition_1)
  participantes_2=participantes_no_confirm(deposition_2)
  return jaccard_similarity(participantes_1,participantes_2)

Segunda variable: $D_{i,j}$

In [0]:
#En este caso tenemos dos situaciones diferentes. Una persona puede estar en un grupo confirmado, lo que indicaría que está en más de una deposition y, por lo tanto, puede que le haya cogido declaración más de un deponent.
#O puede que una persona esté en una única deposition y, por lo tanto, solo tenga un deponent.

#Primero vamos a crear una función que dado un deponent busque a todas las personas a las que ha tomado declaración.

#Entrada: el deponent puesto con su indx o su nombre, indiferente.

def declaracion(deponent):
  if isinstance(deponent,str):
    deponent=person2indx[deponent]
  personas_total=list()
  for deposition in indx2role[deponent].keys():
    if indx2role[deponent][deposition]=='Deponent' or indx2role[deponent][deposition]=='deponent':
      personas=participantes_no_confirm(deposition)
      personas_total=union(personas_total,personas)
  return personas_total

#Lo que devolvemos es una lista con todas las personas a las que les ha cogido testimonio cierto deponent.
  



In [0]:
# Dada una deposition, vamos a buscar su deponent.
# No podemos dar la persona, ya que en caso de coincidir en el nombre con otra que tampoco esté confirmada tendríamos el problema de tener más de un deponent.
def search_deponent(deposition):
  deponent=0
  for persona in depositions_df.iloc[deposition]['people_list']:
    if 'role' in persona.keys():
      if persona['role']=='Deponent':
        deponent=persona['forename']+' '+persona['surname'] #Estamos asumiendo que el deponent tiene siempre 'forename' y 'surname'
      elif persona['role']=='deponent':
        deponent=persona['forename']+' '+persona['surname']
    if deponent!=0:
      break
  return deponent

In [0]:
# En este caso volvemos a tener dos personas que no están confirmadas. Si lo estuviesen tendríamos más de una deposition. Esto habrá que cambiarlo en un futuro también!!!!
def D(deposition_1, deposition_2):
  deponent_1=search_deponent(deposition_1)
  deponent_2=search_deponent(deposition_2)
  if deponent_1==deponent_2: #En caso de que los deponents sean el mismo, su índice de Jaccard sería 1. Vamos a penalizar un poco ese resultado hasta que se nos ocurra uno mejor.
    return 0.8
  else:
    personas_1=declaracion(deponent_1)
    personas_2=declaracion(deponent_2)
    return jaccard_similarity(personas_1,personas_2)

Tercera variable: $A_{i,j}$

In [0]:
##Sacamos cuales son las estructuras más comunes.


In [0]:
#Para esto lo único que hay que hacer es una división, entre el número de cambios por el número de letras del apellido más largo.
#Como lo que queremos es que cuanto más similares sean dos nombres más cercanos estén de 1, entonces tenemos que hacer 1-división.

def Aij(nombre_1,nombre_2):
  if isinistance(nombre_1,int):
    nombre_1=indx2person[nombre_1]
  if isinistance(nombre_2,int):
    nombre_2=indx2person[nombre_2]
  

Cuarta variable: $r_{i,j}$

In [0]:
#Esta función nos devuelve la distancia en kilometros entre dos pueblos (siempre y cuando existan coordenadas).
#Entrada: ahora de momento el índice de la línea en el que viene la deposition.

def distance(town_1,town_2):
  if math.isnan(depositions_df.iloc[town_1]['deponent_town_lat']) or math.isnan(depositions_df.iloc[town_1]['deponent_town_lng']) or math.isnan(depositions_df.iloc[town_2]['deponent_town_lat']) or math.isnan(depositions_df.iloc[town_2]['deponent_town_lng']):
    #raise ValueError('Alguna no tiene suficientes argumentos')
    return 0
  else:
    coords_1=list()
    coords_1.append(depositions_df.iloc[town_1]['deponent_town_lat'])
    coords_1.append(depositions_df.iloc[town_1]['deponent_town_lng'])
    coords_1=tuple(coords_1)
    coords_2=list()
    coords_2.append(depositions_df.iloc[town_2]['deponent_town_lat'])
    coords_2.append(depositions_df.iloc[town_2]['deponent_town_lng'])
    coords_2=tuple(coords_2)
    return geopy.distance.vincenty(coords_1, coords_2).km


Quinta variable: $S_{i,j}$

In [0]:
def person_in_deposition(persona, deposition):
  for i in range(len(depositions_df.iloc[deposition]['people_list'])):
    nombre=depositions_df.iloc[deposition]['people_list'][i]['forename']+' '+depositions_df.iloc[deposition]['people_list'][i]['surname']
    if persona==nombre:
      break
  return depositions_df.iloc[deposition]['people_list'][i]
  

In [0]:
def obtain_sex(persona, deposition):
  dictionary=person_in_deposition(persona,deposition)
  if 'sex' in dictionary.keys():
    return dictionary['sex']
  else:
    return 0


def Sex(persona_1, deposition_1, persona_2, deposition_2):
  if isinstance(persona_1,int):
    persona_1=indx2person[persona_1]
  if isinstance(persona_2,int):
    persona_2=indx2person[persona_2]
  sexo_1=int(obtain_sex(persona_1,deposition_1))
  sexo_2=int(obtain_sex(persona_2, deposition_2))
  if sexo_1==0 or sexo_2==0 or sexo_1==3 or sexo_2==3:
    return 0.8
  elif sexo_1==sexo_2:
    return 1
  else:
    return 0.3
  

Sexta y séptima variables: $R_{i,j}$ y $O_{i,j}$

In [0]:

def obtain_role(persona,deposition):
  if deposition in indx2role[persona].keys():
    return indx2role[persona][deposition]
  else:
    return 0

def Role(persona_1,deposition_1,persona_2,deposition_2):
  if isinstance(persona_1,str):
    persona_1=person2indx[persona_1]
  if isinstance(persona_2, str):
    persona_2=person2indx[persona_2]
  role_1=obtain_role(persona_1,deposition_1)
  role_2=obtain_role(persona_2,deposition_2)
  if role_1==0 or role_2==0:
    return 0.9
  elif role_1==role_2:
    return 1
  else:
    return 0.7
  

In [0]:

###Esto es para la occupation pero es algo más complejo, así que ya lo harás!!!! (Es lo de las diferentes particiones en cuanto nombre y apellido)
def obtain_occupation(persona, deposition):
  dictionary=person_in_deposition(persona,deposition)
  if 'occupation' in dictionary.keys():
    return dictionary['occupation']
  else:
    return 0


def Occupation(persona_1, deposition_1, persona_2, deposition_2):
  if isinstance(persona_1,int):
    persona_1=indx2person[persona_1]
  if isinstance(persona_2,int):
    persona_2=indx2person[persona_2]
  occupation_1=obtain_occupation(persona_1,deposition_1)
  occupation_2=obtain_occupation(persona_2, deposition_2)
  if occupation_1==0 or occupation_2==0:
    return 0.9
  elif occupation_1==occupation_2:
    return 1
  else:
    return 0.7

Octava variable: Gauss

In [0]:
#Esta función nos devuelve la diferencia de días entre depositions

def get_year(deposition_1,deposition_2):
  if isinstance(depositions_df.iloc[deposition_1]['creation_date'],str) and isinstance(depositions_df.iloc[deposition_2]['creation_date'],str):
    date_1=depositions_df.iloc[deposition_1]['creation_date'].split('-')
    date_2=depositions_df.iloc[deposition_2]['creation_date'].split('-')
    diference=date(int(date_1[0]),int(date_1[1]),int(date_1[2]))-date(int(date_2[0]),int(date_2[1]),int(date_2[2]))
    return abs(diference.days)
  else:
    return 200 #Vamos a poner un valor aleatorio de 200 días entre sí.



In [0]:
#Idea importante para Gauss: puedes determinar cuanto consideras que es suficiente tiempo como para que el resto de valores de ahí hacia delante sea 1 a la hora de
#meterlo en la función de Gauss y de ahí para abajo realizar la división anterior.

def Gauss(deposition_1,deposition_2): #En esta definición tengo que sacar la distancia máxima entre dos ciudades (d_max) y el número máximo de días (y_máx.)
  year=1-((get_year(deposition_1,deposition_2))/9661)
  distancia_depositions=distance(deposition_1,deposition_2)/453.32
  return 1-math.exp(-((((year-1)**2)/(1/2))+(((distancia_depositions-1)**2)/(1/2))))

In [0]:
Gauss(4890,471)

Variable final: $M_{i,j}$

$M_{i,j}=(\alpha * Levratio_{i,j}+\beta * Gauss_{i,j}+\gamma * (\gamma_{1}*G_{i,j}+\gamma_{2}*D_{i,j}))*S_{i,j}*R_{i,j}*O_{i,j}$

In [0]:
def M_2(persona_1,deposition_1,persona_2,deposition_2):
  alpha=0.55
  beta=0.35
  gamma=0.1
  gamma_1=0.6
  gamma_2=0.4
  return (alpha*ratio(persona_1,persona_2)+beta*Gauss(deposition_1,deposition_2)+gamma*(gamma_1*GG(deposition_1,deposition_2)+gamma_2*D(deposition_1,deposition_2)))*Sex(persona_1,deposition_1,persona_2,deposition_2)*Role(persona_1,deposition_1,persona_2,deposition_2)*Occupation(persona_1,deposition_1,persona_2,deposition_2)

## Funciones de nx-altair

Funciones de nx-altair con modificaciones

In [0]:
def draw_networkx_edges(
    G=None,
    pos=None,
    chart=None,
    layer=None,
    edgelist=None,
    width=1,
    alpha=1.0,
    edge_color='black',
    edge_cmap=None,
    tooltip=None,
    legend=False,
    **kwargs):
    """Draw the edges of the graph G.
    This draws only the edges of the graph G.
    Parameters
    ----------
    G : graph
       A networkx graph
    pos : dictionary
       A dictionary with nodes as keys and positions as values.
       Positions should be sequences of length 2.
    chart:
    edgelist : collection of edge tuples
       Draw only specified edges(default=G.edges())
    width : float, or array of floats
       Line width of edges (default=1.0)
    edge_color : color string, or array of floats
       Edge color. Can be a single color format string (default='r'),
       or a sequence of colors with the same length as edgelist.
       If numeric values are specified they will be mapped to
       colors using the edge_cmap and edge_vmin,edge_vmax parameters.
    alpha : float
       The edge transparency (default=1.0)
    edge_cmap : Matplotlib colormap
       Colormap for mapping intensities of edges (default=None)
    Returns
    -------
    viz: ``altair.Chart`` object
    """

    if chart is None:
        # Pandas dataframe of edges
        df_edges = to_pandas_edges(G, pos)
        

        # Build a chart
        edge_chart = alt.Chart(df_edges).properties(
    width=600,
    height=600)

    else:
        df_edges = chart.layer[0].data
        edge_chart = chart.layer[0]

    marker_attrs = {}
    encoded_attrs = {}

    # ---------- Handle arguments ------------

    ###### node list argument
    if isinstance(edgelist, list):
        # Subset dataframe.
        df_edges = df_edges.loc[df_edges['pair'].isin(edgelist)]

    elif edgelist is not None:
        raise Exception("nodelist must be a list or None.")


    ###### Node size
    if isinstance(width, str):
        encoded_attrs["size"] = alt.Size(width, scale=alt.Scale(range=[1,4]), legend=None) #sort='descending'

    elif isinstance(width, float) or isinstance(width, int):
        marker_attrs["strokeWidth"] = width

    else:
        raise Exception("width must be a string or int.")

    ###### node_color
    if not isinstance(edge_color, str):
        raise Exception("edge_color must be a string.")

    elif edge_color in df_edges.columns:
        encoded_attrs["color"] =alt.Color(edge_color, legend=None)

    else:
        marker_attrs["color"] = edge_color

    ##### alpha
    if isinstance(alpha, str):
        encoded_attrs["opacity"] = alpha

    elif isinstance(alpha, int) or isinstance(alpha, float):
        marker_attrs["opacity"] = alpha

    elif alpha is not None:
        raise Exception("alpha must be a string or None.")

    ##### alpha
    if isinstance(edge_cmap, str):
        encoded_attrs["color"] = alt.Color(
            edge_color,
            legend=alt.Legend(title="M",orient='top'),
            scale=alt.Scale(scheme=edge_cmap,domain=(0,1)),type='quantitative')

    elif edge_cmap is not None:
        raise Exception("edge_cmap must be a string (colormap name) or None.")

    if tooltip is not None:
        encoded_attrs['tooltip'] = tooltip

    # ---------- Construct visualization ------------

    # Draw edges
    

    #MOD: Ahora generamos las dos visualizaciones, aristas y nodos, teniendo
    # en cuenta que se hace en dos capas diferentes y luego se juntan.
    edge_chart = edge_chart.mark_line(size=50,**marker_attrs).encode(
        alt.X('x',type='quantitative'),
        alt.Y('y', type='quantitative'),
        order=alt.Order('edge',type='nominal'),
        **encoded_attrs
    )

    if chart is not None:
        chart.layer[0] = edge_chart

    return edge_chart #chart_tot

In [0]:
def draw_networkx_edges_mark_bar(
    G=None,
    pos=None,
    chart=None,
    letra='J',
    **kwargs):

    if chart is None:
        # Pandas dataframe of edges
        df_edges = to_pandas_edges(G, pos)
        

        # Build a chart
        edge_chart = alt.Chart(df_edges).properties(
    width=200,
    height=150)
    else:
        df_edges = chart.layer[0].data
        edge_chart = chart.layer[0]

    marker_attrs = {}
    encoded_attrs = {}

    # ---------- Construct visualization ------------

    
    edge_chart = edge_chart.mark_bar(size=12,**marker_attrs).encode(
    x='pair',
    y=alt.Y(letra, scale=alt.Scale(domain=(0,1))),
    color=alt.Color('id:O', legend=None))

    if chart is not None:
        chart.layer[0] = edge_chart

    return edge_chart #chart_tot

In [0]:
def draw_networkx_edges_circle_point(
    G=None,
    pos=None,
    chart=None,
    letra='J',
    **kwargs):
  
    if chart is None:
        # Pandas dataframe of edges
        df_edges = to_pandas_edges(G, pos)
        

        # Build a chart
        edge_chart = alt.Chart(df_edges).properties(
    width=200,
    height=50)
    else:
        df_edges = chart.layer[0].data
        edge_chart = chart.layer[0]

    marker_attrs = {}
    encoded_attrs = {}

    # ---------- Construct visualization ------------

    valor='mean('+letra+')'

    if letra=='Occupation_int':  
      edge_chart = edge_chart.mark_circle(size=100).encode(
      x=alt.X('pair',axis=alt.Axis(grid=False)),
      y=alt.Y('y_point',title='Occupation',scale=alt.Scale(domain=(0,2)),axis=alt.Axis(grid=False)),
      color=alt.Color(valor,legend=None,scale=alt.Scale(domain=(0.3,1))),
      tooltip=alt.Tooltip(letra))
    elif letra=='Sex_int':
      edge_chart = edge_chart.mark_circle(size=100).encode(
      x=alt.X('pair',axis=alt.Axis(grid=False)),
      y=alt.Y('y_point',title='Sex',scale=alt.Scale(domain=(0,2)),axis=alt.Axis(grid=False)),
      color=alt.Color(valor,legend=alt.Legend(title="Scale",orient='left'),scale=alt.Scale(domain=(0.3,1))),
      tooltip=alt.Tooltip(letra))
    else:
      edge_chart = edge_chart.mark_circle(size=100).encode(
      x=alt.X('pair',axis=alt.Axis(grid=False)),
      y=alt.Y('y_point',title='Role',scale=alt.Scale(domain=(0,2)),axis=alt.Axis(grid=False)),
      color=alt.Color(valor,legend=None,scale=alt.Scale(domain=(0.3,1))),
      tooltip=alt.Tooltip(letra))
      
    if chart is not None:
        chart.layer[0] = edge_chart

    return edge_chart

In [0]:
def draw_heatmap_1(
    G=None,
    pos=None,
    letra='J'):
  
  letra_2=letra[:-4]
  
  heatmap=alt.Chart(to_pandas_edges(H,pos)).properties(
    width=100,
    height=100).mark_rect().encode(
    alt.X('target:O',title=None),
    alt.Y('source:O',title=None),
    color=alt.Color(letra,scale=alt.Scale(domain=[0,1])),
    tooltip=alt.Tooltip(letra)
).properties(title=letra_2).interactive()

  return(heatmap)

In [0]:
def draw_heatmap_2(
    G=None,
    pos=None,
    letra='J'):
  
  letra_2=letra[:-4]
  
  heatmap_2=alt.Chart(to_pandas_edges(H,pos)).properties(
    width=100,
    height=100).mark_rect().encode(
    alt.X('source:O',title=None),
    alt.Y('target:O',title=None),
    color=alt.Color(letra,scale=alt.Scale(domain=[0,1])),
    tooltip=alt.Tooltip(letra)
).properties(title=letra_2).interactive()

  return(heatmap_2)

In [0]:
def draw_networkx_slider(
    G=None,
    pos=None,
    chart=None,
    nodelist=None,
    edgelist=None,
    node_size=300,
    node_color='red',
    node_label=None,
    font_color='black',
    font_size=15,
    alpha=1,
    cmap=None,
    width=1,
    arrow_width=2,
    arrow_length=.1,
    edge_color='black',
    arrow_color='black',
    node_tooltip=None,
    edge_tooltip=None,
    edge_cmap=None):
    """Draw the graph G using Altair.
    nodelist : list, optional (default G.nodes())
       Draw only specified nodes
    edgelist : list, optional (default=G.edges())
       Draw only specified edges
    node_size : scalar or array, optional (default=300)
       Size of nodes.  If an array is specified it must be the
       same length as nodelist.
    node_color : color string, or array of floats, (default='r')
       Node color. Can be a single color format string,
       or a  sequence of colors with the same length as nodelist.
       If numeric values are specified they will be mapped to
       colors using the cmap and vmin,vmax parameters.  See
       matplotlib.scatter for more details.
    font_size : scalar or string
       Size of nodes (default=15).  If an array is specified it must be the
       same length as nodelist.
    font_color : color string, or array of floats
       Node color. Can be a single color format string (default='r'),
       or a  sequence of colors with the same length as nodelist.
       If numeric values are specified they will be mapped to
       colors using the cmap and vmin,vmax parameters.  See
       matplotlib.scatter for more details.
    node_label : string
       The name of the node attribute to treat as a label.
    alpha : float, optional (default=1.0)
       The node and edge transparency
    cmap : Matplotlib colormap, optional (default=None)
       Colormap for mapping intensities of nodes
    width : float, optional (default=1.0)
       Line width of edges
    arrow_width : float, optional (default=2.0)
       The width of arrow portions of edges.
    arrow_length : float, optional (default=.1)
       The perportion of the line to be occupied by the arrow.
    edge_color : color string, or array of floats (default='r')
       Edge color. Can be a single color format string,
       or a sequence of colors with the same length as edgelist.
       If numeric values are specified they will be mapped to
       colors using the edge_cmap and edge_vmin,edge_vmax parameters.
    arrow_color : color string, or array of floats (default='r')
       Arrow color. Can be a single color format string,
       or a sequence of colors with the same length as edgelist.
       If numeric values are specified they will be mapped to
       colors using the edge_cmap and edge_vmin,edge_vmax parameters.
    edge_cmap : Matplotlib colormap, optional (default=None)
       Colormap for mapping intensities of edges
    """
    slider=alt.binding_range(min=0,max=1,step=0.05)
    select_M=alt.selection_single(name='RangeMin',fields=['M'], bind=slider, init={'M':0.5})
    select_M_2=alt.selection_single(name='RangeMax',fields=['M'], bind=slider, init={'M':0.9})
    highlightNode = alt.selection_interval(name='brush_fin',empty='all')
    

    # Draw edges
    
    
    if len(G.edges())>0:
      base=alt.Chart(to_pandas_edges(G,pos))
      a=base.mark_line(stroke='grey', strokeOpacity=0.5).encode(
      alt.X('x',type='quantitative'),
      alt.Y('y',type='quantitative'),
      order=alt.Order('edge',type='nominal')).add_selection(highlightNode,select_M,select_M_2).transform_filter(
                "datum.M > RangeMin_M"
                ).transform_filter(
                "datum.M < RangeMax_M"
                )
      b=draw_networkx_edges(G,pos,edgelist=edgelist,
            alpha=alpha,
            width=width,
            edge_color=edge_color,
            edge_cmap=edge_cmap,
            tooltip=edge_tooltip
            )
      b=b.transform_lookup('edge',from_=alt.LookupSelection(selection='brush_fin',key='edge'), as_='highlightEdges').transform_filter('datum.highlightEdges').transform_filter(
                "datum.M > RangeMin_M"
                ).transform_filter(
                "datum.M < RangeMax_M"
                )
      
        #######################################
      edges_radio=alt.layer(draw_heatmap_1(
            G,
            pos,
            letra='radio_int'
            ).transform_filter(
                "datum.M > RangeMin_M"
                ).transform_filter(
                "datum.M < RangeMax_M"
                ).transform_filter( highlightNode ),draw_heatmap_2(
            G,
            pos,
            letra='radio_int'
            ).transform_filter(
                "datum.M > RangeMin_M"
                ).transform_filter(
                "datum.M < RangeMax_M"
                ).transform_filter( highlightNode ))

      edges_D=draw_networkx_edges_mark_bar(
            G,
            pos,
            letra='mean(D_int)').transform_filter(
                "datum.M > RangeMin_M"
                ).transform_filter(
                "datum.M < RangeMax_M"
                ).transform_filter(highlightNode)

      edges_Gauss=draw_networkx_edges_mark_bar(
            G,
            pos,
            letra='mean(Gauss_int)'
            ).transform_filter(
                "datum.M > RangeMin_M"
                ).transform_filter(
                "datum.M < RangeMax_M"
                ).transform_filter(highlightNode)

      edges_G=draw_networkx_edges_mark_bar(
            G,
            pos,
            letra='mean(GG_int)'
            ).transform_filter(
                "datum.M > RangeMin_M"
                ).transform_filter(
                "datum.M < RangeMax_M"
                ).transform_filter(highlightNode)

      edges_Sex=draw_networkx_edges_circle_point(
            G,
            pos,
            letra='Sex_int'
            ).transform_filter(
                "datum.M > RangeMin_M"
                ).transform_filter(
                "datum.M < RangeMax_M"
                ).transform_filter(highlightNode).interactive()

      edges_Role=draw_networkx_edges_circle_point(
            G,
            pos,
            letra='Role_int'
            ).transform_filter(
                "datum.M > RangeMin_M"
                ).transform_filter(
                "datum.M < RangeMax_M"
                ).transform_filter(highlightNode).interactive()

      edges_Occupation=draw_networkx_edges_circle_point(
            G,
            pos,
            letra='Occupation_int'
            ).transform_filter(
                "datum.M > RangeMin_M"
                ).transform_filter(
                "datum.M < RangeMax_M"
                ).transform_filter(highlightNode).interactive()
        ######################################
        

      if isinstance(G, nx.DiGraph):
            # Draw edges
            arrows = draw_networkx_arrows(
                G,
                pos,
                edgelist=edgelist,
                alpha=alpha,
                arrow_width=arrow_width,
                arrow_length=arrow_length,
                edge_color=arrow_color,
                edge_cmap=edge_cmap,
                tooltip=edge_tooltip,
                )

    # Draw nodes
    if len(G.nodes())>0:
        nodes = draw_networkx_nodes(
            G,
            pos,
            nodelist=nodelist,
            node_size=node_size,
            node_color=node_color,
            alpha=alpha,
            cmap=cmap,
            tooltip=node_tooltip,
            brush=highlightNode
        )
        
        # Draw node labels:
        if node_label:
            labels = draw_networkx_labels(
                G,
                pos,
                nodelist=nodelist,
                font_size=font_size,
                font_color=font_color,
                node_label=node_label
            )

    # Layer the chart
        viz = []
        if len(G.edges()):
            viz.append(b)
            if isinstance(G, nx.DiGraph):
              viz.append(arrows)

        if len(G.nodes()):
            viz.append(nodes)
            if node_label:
                viz.append(labels)

        if viz:
            viz = alt.layer(*viz)
            viz=alt.layer(viz)
        else:
            raise ValueError("G does not contain any nodes or edges.")

    # Remove ticks, axis, labels, etc.
        
    return alt.layer(a,b,nodes,labels)|edges_radio&edges_G&edges_Sex|edges_Gauss&edges_D&(edges_Role|edges_Occupation)

In [0]:
# Generate a random graph
H = nx.Graph()

for i in range(839,840):
  indx_1=i
  name_1=indx2person[i]
  for t in range(len(indx2role[i].keys())):
    depo_1_list=list(indx2role[i].keys())
    depo_1=depo_1_list[t]
    for j in range(i,840):
      indx_2=j
      name_2=indx2person[j]
      for l in range(len(indx2role[j].keys())):
        depo_2_list=list(indx2role[j].keys())
        depo_2=depo_2_list[l]
        if i==j and l<=t:
          pass
        else:
          tuple_1=str(i)+str(depo_1)
          tuple_2=str(j)+str(depo_2)
          Ms=M_2(name_1,depo_1,name_2,depo_2)
          radio_int=ratio(name_1,name_2)  
          Gauss_int=Gauss(depo_1,depo_2)
          GG_int=GG(depo_1,depo_2)
          D_int=D(depo_1,depo_2)
          Sex_int=Sex(name_1,depo_1,name_2,depo_2)
          Role_int=Role(name_1,depo_1,name_2,depo_2)
          Occupation_int=Occupation(name_1,depo_1,name_2,depo_2)
          H.add_edge(tuple_1,tuple_2,radio_int=radio_int,M=Ms,Gauss_int=Gauss_int,GG_int=GG_int,D_int=D_int,Sex_int=Sex_int,Role_int=Role_int,Occupation_int=Occupation_int,y_point=1)
pos = nx.kamada_kawai_layout(H,weight='M')

for n in H.nodes():
  H.nodes[n]['name'] = n

In [0]:
draw_networkx_slider(H,pos,node_label='name',node_color='#9ecae1',edge_color='M:N',node_tooltip='name',edge_cmap='yelloworangered',width='M:N')

In [0]:
#Esto está hecho para sacar las personas que tenían más de un sexo diferente asociado
#for i in range(len(indx2role)):
#  dictionary=indx2role[i]
#  nombre=indx2person[i]
#  sexos=list()
#  a=0
#  for key in dictionary.keys():
#    sexo=int(obtain_sex(nombre,key))
#    if sexo==2 or sexo==1 and sexo not in sexos and len(sexos)==0:
#      sexos.append(sexo)
#    elif sexo==2 or sexo==1 and sexo not in sexos and len(sexos)==1:
#      sexos.append(sexo)
#      a=7
#    if a==7:
#      print(i)
#      break


In [0]:
#contador_bien=0
#contador_mal=0
#contador_no_sabe=0
#contador_total=0
#for deposition in range(7):
#  for persona in depositions_df.iloc[deposition]['people_list']:
#    if 'forename' in persona.keys() and 'sex' in persona.keys():
#      nombre=persona['forename']
#      sexo=int(persona['sex'])
#      sexo_estim=gender.Detector().get_gender(nombre)
#      if sexo_estim=='male' or sexo_estim=='mostly_male':
#        sexo_estim=1
#     elif sexo_estim=='female' or sexo_estim=='mostly_female':
#       sexo_estim=2
#      else:
#        sexo_estim=3
#      if sexo==sexo_estim and sexo!=3:
#        contador_bien+=1
#      elif sexo==3:
#       contador_no_sabe+=1
#    elif sexo!=3 and sexo!=sexo_estim:
#        contador_mal+=1

      

      